# Deepchem GCN

## Featurizing molecules

In [1]:
!pip install --pre deepchem
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 11.8 MB/s 
     |████████████████████████████████| 36.8 MB 79.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 12.2 MB/s 
     |████████████████████████████████| 181 kB 54.2 MB/s 
     |████████████████████████████████| 158 kB 43.1 MB/s 
     |████████████████████████████████| 63 kB 550 kB/s 
     |████████████████████████████████| 157 kB 60.8 MB/s 
     |████████████████████████████████| 157 kB 61.6 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████████| 157 kB 61.9 MB/s 
     |████████████████████████████████| 157 kB 66.8 MB/s 
     |████████████████████████████████| 157 kB 60.5 MB/s 
     |████████████████████████████████| 157 kB 62.0 MB/s 
     |████████████████████████████████| 156 kB 67

In [2]:
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps

import os
from pathlib import Path
import csv
import math

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
content_path = Path('/content/drive/MyDrive/chem_data')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -qU deepchem wandb

In [4]:
import wandb
wandb.login(key="d3b428cbc87d39cfd351c091f1a76cb141d496f4")
# wandb.login(key="d1f13b428cbc87d39cfd351c09a76cb141d496f4")

# import warnings
# warnings.filterwarnings('ignore')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
wandb.login()

wandb: Currently logged in as: venustheplanet. Use `wandb login --relogin` to force relogin


True

In [6]:
from deepchem.models.wandblogger import WandbLogger

In [7]:
# Create directories
# def create_dir():
""" Create directories ."""
custom_gcn_dir = content_path/'custom_gcn_dir'
if not custom_gcn_dir.is_dir():
  custom_gcn_dir.mkdir()

# create_dir()
basic_gcn_dir = content_path/'basic_gcn_dir'
if not basic_gcn_dir.is_dir():
  basic_gcn_dir.mkdir()

gcn_dir = content_path/'gcn_dir'
if not gcn_dir.is_dir():
  gcn_dir.mkdir()

In [8]:
# converting sdf files into csv files consisting of smiles and labels
def sdf_to_csv(filename):
  """ converts sdf files to csv files """
  # Check if file exist
  file_exist = Path(f"{content_path}/{filename}.csv")
  if not file_exist.is_file():
    print('converting sdf file to csv')

    supp = Chem.SDMolSupplier(f"{content_path}/{filename}.sdf")
    mols = [x for x in supp]
    # pki_values = [-1*math.log10(float(x.GetProp("pKi"))*(10^-9)) for x in supp]
    pki_values = []
    for x in supp:
      value = float(x.GetProp("pKi"))
      log_val = -1 * math.log10(value * (1e-9))
      pki_values.append(log_val)
    assert len(pki_values) == len(mols)
    smiles = [Chem.MolToSmiles(m) for m in mols]
    # create csv
    data_dict = {'smiles': smiles, 'pki': pki_values}
    data_df = pd.DataFrame(data_dict)
    data_df.to_csv(f"{content_path}/{filename}.csv")
  
  else:
    # print('file alr exist')
    None

In [9]:
# Featurize data into dc dataset
def csv_to_feat(filename, feat="mol"):

  data = pd.read_csv(f"{content_path}/{filename}.csv")
  smiles = data['smiles'].tolist()
  pki = data['pki'].tolist()
  if feat == "mol":
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
  elif feat == "gcn":
    featurizer = dc.feat.ConvMolFeaturizer()
  X = featurizer.featurize(smiles)
  dataset = dc.data.NumpyDataset(X=X, y=pki)
  # print(dataset)
  # print(pki)
  return dataset

In [10]:
# Metrics

metric = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

pearson_r2_score = dc.metrics.Metric(
    dc.metrics.pearson_r2_score, task_averager=np.mean, mode="regression")

mean_absolute_error = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

rms_score = dc.metrics.Metric(
    dc.metrics.rms_score, task_averager=np.mean, mode="regression")

### Trial GCNModel

In [ ]:
!pip install dgllife
# !pip install dgl
!pip install dgl-cu101 dglgo -f https://data.dgl.ai/wheels/repo.html

In [12]:
from deepchem.models import GCNModel
from deepchem.models.wandblogger import WandbLogger
from deepchem.models import GraphConvModel

In [29]:
# from wandb.sdk.wandb_init import wandb_login
# train custom model
def train_custom_gcn(train, val, tid, num_layers=2, num_epoch=30, dense_layer_size=128, num_layer_size=64):
  graph_conv_layers=[num_layer_size]*num_layers
  print(graph_conv_layers)
  # callback = dc.models.ValidationCallback(val, 500, metric)
  logger = WandbLogger(project='gcn_custom', name=tid)
  model = GraphConvModel(mode='regression', 
                         n_tasks=1, batch_size=16, 
                         learning_rate=0.001, 
                         graph_conv_layers=graph_conv_layers, 
                        #  dropout=0.5, 
                        #  dense_layer_size=dense_layer_size,
                         wandb_logger=logger)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.01, graph_conv_layers=[64]*num_layers, , wandb_logger=logger)
  loss = model.fit(train, nb_epoch=num_epoch)
  # loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model

In [33]:
def explore_num_layers_gcn(data):
  num_layers = [2, 4, 6, 8, 10]
  mae_list = []
  r2_list = []
  rmse_list = []
  tid = 'tid' + data
  print(tid)
  sdf_to_csv(tid)
  dataset = csv_to_feat(tid,)
  train, val, test = split_data(dataset)
  for i in num_layers: 
    model = train_GCN(train, val, tid, num_layers=i)
    mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
    r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
    rmse = model.evaluate(test, [rms_score], per_task_metrics=False)
    print(f"mae: {mae}")
    print(f"r2: {r2}")
    print(f"rmse: {rmse}")
    mae_list.append(mae['mean-mean_absolute_error'])
    r2_list.append(r2['mean-pearson_r2_score'])
    rmse_list.append(rmse['mean-rms_score'])
  print(mae_list)
  print(r2_list)
  print(rmse_list)

  return num_layers, mae_list, r2_list, rmse_list

In [36]:
explore_num_layers_gcn('15')

tid15
Step 1000 validation: mean-mean_absolute_error=0.682748


eval/mean-mean_absolute_error_(0),▁
train/loss,▁▁▁▁▁▁▁▁▁▁█▃▂▁▂▂▁▁
eval/mean-mean_absolute_error_(0),0.68275
train/loss,1.73891


mae: {'mean-mean_absolute_error': 1.2265777348890805}
r2: {'mean-pearson_r2_score': 0.04236934442497331}
rmse: {'mean-rms_score': 1.5728435347014131}
mae: {'mean-mean_absolute_error': 1.2926011465206901}
r2: {'mean-pearson_r2_score': 0.032206427008865335}
rmse: {'mean-rms_score': 1.5952976356560338}


Step 1000 validation: mean-mean_absolute_error=0.683631


eval/mean-mean_absolute_error_(0),▁
train/loss,▁▁▁▁▁▁▁▁▁▁█▃▂▂▁▁▁▁
eval/mean-mean_absolute_error_(0),0.68363
train/loss,32.33993


...

mae: {'mean-mean_absolute_error': 0.8208248504784575}
r2: {'mean-pearson_r2_score': 0.0056642413886007215}
rmse: {'mean-rms_score': 1.0126455041551439}
mae: {'mean-mean_absolute_error': 0.7934980540518044}
r2: {'mean-pearson_r2_score': 0.07949223557596735}
rmse: {'mean-rms_score': 0.9810646544477618}


Step 1000 validation: mean-mean_absolute_error=0.780465


eval/mean-mean_absolute_error_(0),▁
train/loss,▁▁▁▁▁▁▁▁▁▁██▇▇███▇
eval/mean-mean_absolute_error_(0),0.78047
train/loss,35.49178


mae: {'mean-mean_absolute_error': 5.961301841350503}
r2: {'mean-pearson_r2_score': 0.001198052756635132}
rmse: {'mean-rms_score': 6.042544537488703}
mae: {'mean-mean_absolute_error': 5.91779391218469}
r2: {'mean-pearson_r2_score': 0.0008222573781970193}
rmse: {'mean-rms_score': 6.003166143030613}
[1.2926011465206901, 2.1350596981070704, 60.52002635821556, 0.7934980540518044, 5.91779391218469]
[0.032206427008865335, 0.1268852216338657, 0.09187521999888158, 0.07949223557596735, 0.0008222573781970193]
[1.5952976356560338, 2.9984978041982306, 65.2040487206403, 0.9810646544477618, 6.003166143030613]


([2, 4, 6, 8, 10],
 [1.2926011465206901,
  2.1350596981070704,
  60.52002635821556,
  0.7934980540518044,
  5.91779391218469],
 [0.032206427008865335,
  0.1268852216338657,
  0.09187521999888158,
  0.07949223557596735,
  0.0008222573781970193],
 [1.5952976356560338,
  2.9984978041982306,
  65.2040487206403,
  0.9810646544477618,
  6.003166143030613])

In [38]:
num_layers, mae_list, r2_list, rmse_list = explore_num_layers_gcn('136')

tid136


Step 1000 validation: mean-mean_absolute_error=6.66859


eval/mean-mean_absolute_error_(0),▁
train/loss,▃▁▆▅▅▃▄▂▅▁▃▃▁▃▅▄█▅
eval/mean-mean_absolute_error_(0),6.66859
train/loss,1.33087


mae: {'mean-mean_absolute_error': 0.8957072418798753}
r2: {'mean-pearson_r2_score': 0.33846240996371646}
rmse: {'mean-rms_score': 1.0596272959164255}
mae: {'mean-mean_absolute_error': 0.9336224609797285}
r2: {'mean-pearson_r2_score': 0.2205678642879212}
rmse: {'mean-rms_score': 1.108269938965351}


Step 1000 validation: mean-mean_absolute_error=1.8366


...

eval/mean-mean_absolute_error_(0),▁
train/loss,▁▅▄▄▄▇▁▆▅▄▇▃▆█▅▇▃▆
eval/mean-mean_absolute_error_(0),1.05875
train/loss,1.61002


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


mae: {'mean-mean_absolute_error': 1.0577698701605898}
r2: {'mean-pearson_r2_score': nan}
rmse: {'mean-rms_score': 1.2614791174009166}


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


mae: {'mean-mean_absolute_error': 1.047231434249635}
r2: {'mean-pearson_r2_score': nan}
rmse: {'mean-rms_score': 1.2645885831311325}
[0.9336224609797285, 1.0033596650318248, 2595.7261413101455, 3.23165442108849, 1.047231434249635]
[0.2205678642879212, 0.0402536903894398, 0.016483082919702793, 0.057108627112797954, nan]
[1.108269938965351, 1.2239765338084294, 2638.5407508480876, 4.347264008928666, 1.2645885831311325]


In [39]:
num_layers, mae_list, r2_list, rmse_list = explore_num_layers_gcn('15')

tid15


Step 1000 validation: mean-mean_absolute_error=0.601072


eval/mean-mean_absolute_error_(0),▁
train/loss,▃▆▃▂▃▃▅▂▄▂█▃▄▇▃▂▁▃
eval/mean-mean_absolute_error_(0),0.60107
train/loss,0.77793


mae: {'mean-mean_absolute_error': 0.6748992527171797}
r2: {'mean-pearson_r2_score': 0.2604521240969131}
rmse: {'mean-rms_score': 0.8540078956403531}
mae: {'mean-mean_absolute_error': 0.7988402451430975}
r2: {'mean-pearson_r2_score': 0.15123653272783244}
rmse: {'mean-rms_score': 0.9870887136592144}


Step 1000 validation: mean-mean_absolute_error=0.649154


...

eval/mean-mean_absolute_error_(0),▁
train/loss,▂▂▁▂█▃▁▂▅▅▄▅▃▄▄▄▄▃
eval/mean-mean_absolute_error_(0),0.81567
train/loss,0.95593


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


mae: {'mean-mean_absolute_error': 0.8218942666579457}
r2: {'mean-pearson_r2_score': nan}
rmse: {'mean-rms_score': 0.9747301871958974}


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


mae: {'mean-mean_absolute_error': 0.8991564956365757}
r2: {'mean-pearson_r2_score': nan}
rmse: {'mean-rms_score': 1.0345203917575536}
[0.7988402451430975, 0.9717105106598365, 0.8915576016035779, 953982405041345.4, 0.8991564956365757]
[0.15123653272783244, 0.047275917152004174, 0.107556840408997, 0.0033891380832997238, nan]
[0.9870887136592144, 1.1991045465552985, 1.30613691105626, 5086727771724770.0, 1.0345203917575536]


In [26]:
  def explore_num_layers(data):
    num_layers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    mae_list = []
    r2_list = []
    rmse_list = []
    tid = 'tid' + data
    print(tid)
    sdf_to_csv(tid)
    dataset = csv_to_feat(tid, 'gcn')
    train, val, test = split_data(dataset)
    for i in num_layers: 
      model = train_custom_gcn(train, val, tid, num_layers=i)
      mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
      r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
      rmse = model.evaluate(test, [rms_score], per_task_metrics=False)
      print(f"mae: {mae}")
      print(f"r2: {r2}")
      print(f"rmse: {rmse}")
      mae_list.append(mae['mean-mean_absolute_error'])
      r2_list.append(r2['mean-pearson_r2_score'])
      rmse_list.append(rmse['mean-rms_score'])
    print(mae_list)
    print(r2_list)
    print(rmse_list)

    return num_layers, mae_list, r2_list, rmse_list
    # print(model_mae, model_r2, model_rmse)

In [ ]:
num_layers, mae_list, r2_list, rmse_list = explore_num_layers('15')

In [25]:
print(num_layers)
print(mae_list)
print(r2_list)
print(rmse_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0.5644872805556379, 0.6057100016508761, 0.5631045516267449, 0.5748942066505265, 0.5443496476722692, 0.6297650058035911, 0.5660274753006226, 0.5687935787718115, 0.6905683854791222, 0.584643895143269]
[0.46083029764352484, 0.4095661021918552, 0.5696528450157303, 0.4972389570131323, 0.44373968229385663, 0.44594439114435824, 0.5142939095152128, 0.46145771998890267, 0.393168799314881, 0.49464030458120173]
[0.7230652976031006, 0.7646869286818042, 0.7059580375814029, 0.7182331884662408, 0.7256417519693192, 0.8009083906619741, 0.6822940170647995, 0.7268960954036644, 0.8951486630648285, 0.7161433556516059]


In [37]:
# train model model 
def train_GCN(train, val, tid, num_layers=2, num_epoch=30):

  gcn_layers = [1024]*num_layers
  callback = dc.models.ValidationCallback(val, 1000, metric)
  logger = WandbLogger(project='gcn_dc', name=tid)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.001, wandb_logger=logger)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.001, graph_conv_layers=[2048]*num_layers, wandb_logger=logger)
  model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.01, graph_conv_layers=gcn_layers , wandb_logger=logger)
  # loss = model.fit(train, nb_epoch=num_epoch)
  loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model
# return model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']

In [21]:
# split dataset
def split_data(dataset):
  from deepchem.splits.splitters import RandomSplitter

  splitter = RandomSplitter()
  train, val, test = splitter.train_valid_test_split(
      dataset=dataset,
      log_every_n=100
  )
  return train, val, test


In [ ]:
def custom_gcn(data):
  tid = 'tid' + data
  print(tid)
  sdf_to_csv(tid)
  dataset = csv_to_feat(tid, 'gcn')
  train, val, test = split_data(dataset)
  model = train_custom_gcn(train, val, tid)
  model_mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(test, [rms_score], per_task_metrics=False)

  print(model_mae, model_r2, model_rmse)

  return model_mae, model_r2, model_rmse

## Hyperopt

In [ ]:
from hyperopt import hp, fmin, tpe, Trials

In [ ]:
search_space = {
    'layer_sizes': hp.choice('layer_sizes',[[64], [128], [256],[512], [1024], [2048]]),
    # 'num_layers': hp.choice('num_layers',[1, 2, 3, 4, 5, 6]),
    'dense_layer_size': hp.uniform('dense_layer_size', low=64, high=2048),
    'batch_size': hp.uniform('batch_size', low=8, high=200),
    'dropout': hp.uniform('dropout', low=0.0, high=0.8),
    'learning_rate': hp.uniform('learning_rate', low=0.0001, high=0.1),
}

In [ ]:
import tempfile
#tempfile is used to save the best checkpoint later in the program.

metric = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, mode="regression")
    # dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

def fm(args):
  save_dir = tempfile.mkdtemp()
  model = GraphConvModel(n_tasks=1,mode='regression',
                           graph_convlayers=args['layer_sizes']*2, 
                          #  graph_convlayers=args['num_layers']*args['layer_sizes'], 
                           dense_layer_size=int(round(args['dense_layer_size'])),
                         dropout=args['dropout'],
                         learning_rate=args['learning_rate'],
                         batch_size=int(round(args['batch_size'])))
  #validation callback that saves the best checkpoint, i.e the one with the maximum score.
  validation=dc.models.ValidationCallback(val, 100, [metric],save_dir=save_dir,save_on_minimum=False)
  
  model.fit(train, nb_epoch=40,callbacks=validation)

  #restoring the best checkpoint and passing the negative of its validation score to be minimized.
  model.restore(model_dir=save_dir)
  valid_score = model.evaluate(val, [metric], per_task_metrics=False)
  # valid_score = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)

  return valid_score['mean_absolute_error']
  # return valid_score['mean-mean_absolute_error']

In [ ]:
dataset = csv_to_feat('tid15', 'gcn')
train, test, val = split_data(dataset)

In [ ]:
trials=Trials()
best = fmin(fm,
    		space= search_space,
    		algo=tpe.suggest,
    		max_evals=40,
    		trials = trials)

In [ ]:
print(best)

{'batch_size': 144.55044159589042, 'dense_layer_size': 162.00308716637733, 'dropout': 0.6965040650648803, 'layer_sizes': 4, 'learning_rate': 0.08508658568056497}


In [ ]:
custom_gcn('15')

NameError: ignored

## Continue training

In [ ]:
def basic_gcn(data):
  tid = 'tid' + data
  print(tid)
  sdf_to_csv(tid)
  dataset = csv_to_feat(tid, 'mol')
  train, val, test = split_data(dataset)
  model = train_GCN(train, val, tid, 2, 80)
  model_mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(test, [rms_score], per_task_metrics=False)

  return model_mae, model_r2, model_rmse


In [ ]:
basic_gcn('11')

tid11


Step 1000 validation: mean-mean_absolute_error=0.576366
Step 2000 validation: mean-mean_absolute_error=0.861194


eval/mean-mean_absolute_error_(0),▁█
train/loss,█▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▂▁
eval/mean-mean_absolute_error_(0),0.86119
train/loss,1.25663


mae: {'mean-mean_absolute_error': 0.744279892771556}
r2: {'mean-pearson_r2_score': 0.1201473725914473}
rmse: {'mean-rms_score': 0.9982036161015141}


({'mean-mean_absolute_error': 0.8022298934009157},
 {'mean-pearson_r2_score': 0.13652004345803492},
 {'mean-rms_score': 1.034629186246776})

In [ ]:
dataset_ids = ['11', '15', '51', '72', '87', '100', '107', '108', '114', '121', '129', '130', '136', '137', '138', '155', '165', '176', '194', '252', '259', '278', '280', '10142', '10193', '10280', '10627', '11290', '12209', '12952', '19905']
len(dataset_ids)

31

In [ ]:
def run_custom_gcn(num_runs, datasets):
  # results = {}
  mean_results = {}

  for id in datasets:
    results = {}

    mae_list = []
    r2_list = []
    rmse_list = []

    tid = 'tid'+id
    sdf_to_csv(tid)
    dataset = csv_to_feat(tid, 'gcn')
    # train, val, test = split_data(dataset)
    
    for run in range(num_runs):
      train, val, test = split_data(dataset)
      model = train_custom_gcn(train, val, tid)

      mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
      r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
      rmse = model.evaluate(test, [rms_score], per_task_metrics=False)


      print(f"mae: {mae}")
      print(f"r2: {r2}")
      print(f"rmse: {rmse}")
      mae_list.append(mae['mean-mean_absolute_error'])
      r2_list.append(r2['mean-pearson_r2_score'])
      rmse_list.append(rmse['mean-rms_score'])

    results[f'{tid}_mae'] = mae_list
    results[f'{tid}_r2'] = r2_list
    results[f'{tid}_rmse'] = rmse_list

    mean_mae = np.mean(mae_list)
    mean_r2 = np.mean(r2_list)
    mean_rmse = np.mean(rmse_list)

    mean_results[f'{tid}_mean_mae'] = mean_mae
    mean_results[f'{tid}_mean_r2'] = mean_r2
    mean_results[f'{tid}_mean_rmse'] = mean_rmse


    print(f'mean mae for {num_runs} for {tid}: {mean_mae}')
    print(f'mean r2 for {num_runs} for {tid}: {mean_r2}')
    print(f'mean rmse for {num_runs} for {tid}: {mean_rmse}')
  
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{custom_gcn_dir}/{tid}_custom_gcn.csv")

  return results, mean_results

# model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']

In [ ]:
results_custom, mean_results_custom = run_custom_gcn(3, dataset_ids)

In [ ]:
def run_basic_gcn(num_runs, datasets, num_layers, num_epochs):
  # results = {}
  mean_results = {}

  for id in datasets:
    results = {}

    mae_list = []
    r2_list = []
    rmse_list = []

    tid = 'tid'+id
    sdf_to_csv(tid)
    dataset = csv_to_feat(tid)
    # train, val, test = split_data(dataset)
    
    for run in range(num_runs):
      train, val, test = split_data(dataset)
      model = train_GCN(train, val, tid, num_layers, num_epochs)

      mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
      r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
      rmse = model.evaluate(test, [rms_score], per_task_metrics=False)


      print(f"mae: {mae}")
      print(f"r2: {r2}")
      print(f"rmse: {rmse}")
      mae_list.append(mae['mean-mean_absolute_error'])
      r2_list.append(r2['mean-pearson_r2_score'])
      rmse_list.append(rmse['mean-rms_score'])


    
    results[f'{tid}_mae'] = mae_list
    results[f'{tid}_r2'] = r2_list
    results[f'{tid}_rmse'] = rmse_list

    mean_mae = np.mean(mae_list)
    mean_r2 = np.mean(r2_list)
    mean_rmse = np.mean(rmse_list)

    mean_results[f'{tid}_mean_mae'] = mean_mae
    mean_results[f'{tid}_mean_r2'] = mean_r2
    mean_results[f'{tid}_mean_rmse'] = mean_rmse


    print(f'mean mae for {num_runs} for {tid}: {mean_mae}')
    print(f'mean r2 for {num_runs} for {tid}: {mean_r2}')
    print(f'mean rmse for {num_runs} for {tid}: {mean_rmse}')
  
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{gcn_dir}/{tid}_gcn.csv")

  return results, mean_results


In [ ]:
gcn_results, mean_gcn_results = run_basic_gcn(2, dataset_ids, 2, 80)

Step 1000 validation: mean-mean_absolute_error=1.24384
Step 2000 validation: mean-mean_absolute_error=0.686265


eval/mean-mean_absolute_error_(0),█▁
train/loss,█▂▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▂▁▁▁▁▁▁
eval/mean-mean_absolute_error_(0),0.68627
train/loss,0.92642


mae: {'mean-mean_absolute_error': 0.6521984595536973}
r2: {'mean-pearson_r2_score': 0.4629372071679227}
rmse: {'mean-rms_score': 0.8153424422563413}
mae: {'mean-mean_absolute_error': 0.7183164061634397}
r2: {'mean-pearson_r2_score': 0.445082137260992}
rmse: {'mean-rms_score': 0.926716991640502}


Step 1000 validation: mean-mean_absolute_error=1.69692
Step 2000 validation: mean-mean_absolute_error=0.86625


eval/mean-mean_absolute_error_(0),█▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁
eval/mean-mean_absolute_error_(0),0.86625
train/loss,0.96636


...

eval/mean-mean_absolute_error_(0),█▁
train/loss,█▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/mean-mean_absolute_error_(0),0.80326
train/loss,0.78358


mae: {'mean-mean_absolute_error': 0.7308946558544873}
r2: {'mean-pearson_r2_score': 0.03257182259546665}
rmse: {'mean-rms_score': 0.9352152821134123}
mae: {'mean-mean_absolute_error': 0.6181355220360145}
r2: {'mean-pearson_r2_score': 0.11987950056908789}
rmse: {'mean-rms_score': 0.7811376684321325}


Step 1000 validation: mean-mean_absolute_error=0.762554
Step 2000 validation: mean-mean_absolute_error=0.688394


eval/mean-mean_absolute_error_(0),█▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/mean-mean_absolute_error_(0),0.68839
train/loss,0.40405


mae: {'mean-mean_absolute_error': 0.660393319351379}
r2: {'mean-pearson_r2_score': 0.3602449814516183}
rmse: {'mean-rms_score': 0.8272010519691964}
mae: {'mean-mean_absolute_error': 0.5660997734771426}
r2: {'mean-pearson_r2_score': 0.2503998965403067}
rmse: {'mean-rms_score': 0.7373371276727785}
mean mae for 2 for tid19905: 0.5921176477565786
mean r2 for 2 for tid19905: 0.1851396985546973
mean rmse for 2 for tid19905: 0.7592373980524555


In [ ]:
tuned_results, mean_results = run_basic_gcn(2, ['11', '15', '51'], 2, 80)

Step 1000 validation: mean-mean_absolute_error=1.10636
Step 2000 validation: mean-mean_absolute_error=1.01827


eval/mean-mean_absolute_error_(0),█▁
train/loss,▁▁▁▁▁▁▂▁▁▁█▂▃▂▂▃▂▁▁▂▂▁▂▂▂▁
eval/mean-mean_absolute_error_(0),1.01827
train/loss,1.09286


mae: {'mean-mean_absolute_error': 1.0019703506495006}
r2: {'mean-pearson_r2_score': 0.0057583212845425}
rmse: {'mean-rms_score': 1.2473461613219348}


Step 1000 validation: mean-mean_absolute_error=0.9379
Step 2000 validation: mean-mean_absolute_error=0.965694


eval/mean-mean_absolute_error_(0),▁█
train/loss,▃▁▂▃▆▃▂▅▁▂▅▆▄▂▂▄▃▄█▃▂▄▄▅▂▃
eval/mean-mean_absolute_error_(0),0.96569
train/loss,1.1983


...

eval/mean-mean_absolute_error_(0),█▅▃▁▆
train/loss,▅▅▂▅▄▄▄▅▅▆█▇▁▃▅▅▄▄▄▄▄▅▅▄▄▁▃▆▄▅▇▄▃▃▅▄▆▁▄▄
eval/mean-mean_absolute_error_(0),0.7061
train/loss,0.65025


mae: {'mean-mean_absolute_error': 0.5848393663506851}
r2: {'mean-pearson_r2_score': 0.34673924230209185}
rmse: {'mean-rms_score': 0.7752621021576485}


Step 1000 validation: mean-mean_absolute_error=1.39248
Step 2000 validation: mean-mean_absolute_error=0.732231
Step 3000 validation: mean-mean_absolute_error=0.721804
Step 4000 validation: mean-mean_absolute_error=0.723686
Step 5000 validation: mean-mean_absolute_error=0.750211


eval/mean-mean_absolute_error_(0),█▁▁▁▁
train/loss,▄▄▄▇▃▆▂▃▆▄▆▄▁▄▄▄▅▇▃▄▃▄▃▅▅▁▅▅▄▅▆█▆▃▃▄▄▁▃▇
eval/mean-mean_absolute_error_(0),0.75021
train/loss,1.34199


mae: {'mean-mean_absolute_error': 0.777799395464333}
r2: {'mean-pearson_r2_score': 0.30477144604081513}
rmse: {'mean-rms_score': 0.941667359824421}
mean mae for 2 for tid51: 0.6813193809075091
mean r2 for 2 for tid51: 0.32575534417145346
mean rmse for 2 for tid51: 0.8584647309910347


In [ ]:
basic_gcn_results, mean_basic_gcn_results = run_basic_gcn(3, dataset_ids)

### Hyperparameter

In [ ]:
# Hyperparameters tuning
!pip install pyGPGO
import pyGPGO
from pyGPGO.covfunc import matern32
from pyGPGO.acquisition import Acquisition
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.GPGO import GPGO



In [ ]:

dataset = csv_to_feat('tid15')
train, val, test = split_data(dataset)

In [ ]:
# Try Grid hyperoprt training

params_dict = {
    'graph_conv_layers': [[256*2], [512*2], [512*4]],
    'dropouts': [0.2, 0.5],
    'learning_rate': [0.001, 0.0001]
}
optimizer = dc.hyper.GridHyperparamOpt(lambda **p: GCNModel(n_tasks=1, **p))
best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
        params_dict, train, val, metric, nb_epoch=20)

In [ ]:
print(best_hyperparams)

{'graph_conv_layers': [512], 'dropouts': 0.5, 'learning_rate': 0.001}


In [ ]:
import pprint 
pprint.pprint(all_results)

In [ ]:

def grid_hyperparam(param_dict, name):
  logger = WandbLogger(project='gcn_deepchem', name=name)
  optimizer = dc.hyper.GridHyperparamOpt(lambda **p: GCNModel(n_tasks=1, wandb_logger=logger, **p))
  best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
        params_dict, train, val, metric, nb_epoch=20)
  logger.finish()
  return best_model, best_hyperparams, all_results

In [ ]:
params_dict = {
    'graph_conv_layers': [[256*2], [512*2], [512*4]],
    'dropouts': [0.2, 0.5],
    'learning_rate': [0.001, 0.0001]
}

model, hyperparams, all_results = grid_hyperparam(params_dict, 'hyperparam_1layer')

train/loss,▁█
train/loss,37.61662


In [ ]:
pprint.pprint(all_results)

{'_dropouts_0.200000_graph_conv_layers[1024]_learning_rate_0.000100': 7.224334219457447,
 '_dropouts_0.200000_graph_conv_layers[1024]_learning_rate_0.001000': 0.5336553559212444,
 '_dropouts_0.200000_graph_conv_layers[2048]_learning_rate_0.000100': 5.955488099169064,
 '_dropouts_0.200000_graph_conv_layers[2048]_learning_rate_0.001000': 0.8277042890803028,
 '_dropouts_0.200000_graph_conv_layers[512]_learning_rate_0.000100': 7.328100339175753,
 '_dropouts_0.200000_graph_conv_layers[512]_learning_rate_0.001000': 0.8204970570799468,
 '_dropouts_0.500000_graph_conv_layers[1024]_learning_rate_0.000100': 7.283560464725648,
 '_dropouts_0.500000_graph_conv_layers[1024]_learning_rate_0.001000': 0.5576181514744439,
 '_dropouts_0.500000_graph_conv_layers[2048]_learning_rate_0.000100': 7.729851095267512,
 '_dropouts_0.500000_graph_conv_layers[2048]_learning_rate_0.001000': 1.5920897389821194,
 '_dropouts_0.500000_graph_conv_layers[512]_learning_rate_0.000100': 6.665526181446253,
 '_dropouts_0.50000

In [ ]:
print(hyperparams)

{'graph_conv_layers': [2048], 'dropouts': 0.5, 'learning_rate': 0.0001}


In [ ]:
params_dict = {
    'graph_conv_layers': [[512*2], [512*4], [512*5], [2048, 2048], [2048, 2048, 2048]],
    'dropouts': [0.5],
    'learning_rate': [0.0001]
}

model, hyperparams, all_results = grid_hyperparam(params_dict, 'hyperparam_1layer')

train/loss,█▁
train/loss,36.69931


In [ ]:
pprint.pprint(all_results)
print(hyperparams)

{'_dropouts_0.500000_graph_conv_layers[1024]_learning_rate_0.000100': 6.605124628890995,
 '_dropouts_0.500000_graph_conv_layers[2048, 2048, 2048]_learning_rate_0.000100': 5.626514176870728,
 '_dropouts_0.500000_graph_conv_layers[2048, 2048]_learning_rate_0.000100': 7.5665549417123605,
 '_dropouts_0.500000_graph_conv_layers[2048]_learning_rate_0.000100': 6.544542419352204,
 '_dropouts_0.500000_graph_conv_layers[2560]_learning_rate_0.000100': 7.068411053481056}
{'graph_conv_layers': [2048, 2048], 'dropouts': 0.5, 'learning_rate': 0.0001}


In [ ]:
params_dict = {
    'graph_conv_layers': [[2048, 2048], [1024*3, 1024*3],],
    'dropouts': [0.5],
    'learning_rate': [0.0001],
    'batchnorm': [True, False]
}

model, hyperparams, all_results = grid_hyperparam(params_dict, 'hyperparam_1layer')

train/loss,█▁
train/loss,35.31403


In [ ]:
print(hyperparams)
pprint.pprint(all_results)

{'graph_conv_layers': [2048, 2048], 'dropouts': 0.5, 'learning_rate': 0.0001, 'batchnorm': False}
{'_batchnorm_False_dropouts_0.500000_graph_conv_layers[2048, 2048]_learning_rate_0.000100': 7.595424437851707,
 '_batchnorm_False_dropouts_0.500000_graph_conv_layers[3072, 3072]_learning_rate_0.000100': 4.776740861829692,
 '_batchnorm_True_dropouts_0.500000_graph_conv_layers[2048, 2048]_learning_rate_0.000100': 6.645804487216686,
 '_batchnorm_True_dropouts_0.500000_graph_conv_layers[3072, 3072]_learning_rate_0.000100': 7.003691763685035}


In [ ]:
# Try hyperopt bayesian 


params_dict = {
    'batch_size': ('int', [10, 100]),
    'dropout': ('cont', [[.0, 0.5]]),
    'graph_conv_layers': ('int', [32, 2048])
}
params_dict = {
    'graph_conv_layers': 128,
    'dropouts': [0.0, 0.1, 0.2, 0.5],
    'learning_rate': [0.01, 0.001, 0.0001]
}

num_layers = 1
logger = WandbLogger(project='gcn_deepchem', name='trial_hyperopt')

model = GCNModel(mode='regression', n_tasks=1, 
                         batch_size=16, learning_rate=0.01, 
                         graph_conv_layers=[64]*num_layers, wandb_logger=logger)
callback = dc.models.ValidationCallback(val, 1000, metric)

optimizer = dc.hyper.GaussianProcessHyperparamOpt(lambda **p: GCNModel(n_tasks=1, **p))

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
    params_dict, train, val, metric)



Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


LinAlgError: ignored

In [ ]:
params_dict = {
'dense_layer_size': ('int', [16, 2048]),
'graph_conv_layer_size': ('int', [32, 2048]),
'dropout': ('cont', [.0 ,0.5]),
'batch_size': ('int', [10,100]),
'learning_rate': ('cont', [0.0001, 0.0020]),
'nb_epoch': ('int', [20, 200])
}

In [ ]:
df_param = pd.DataFrame.from_dict(params_dict,orient='index', )
display(df_param)

In [ ]:
def hyper_model(dense_layer_size, graph_conv_layer_size, batch_size, dropout, nb_epoch, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    dense_layer_size = int(round(dense_layer_size))
    graph_conv_depth = 2 # two graph convolutional layers
    graph_conv_layer_size = int(round(graph_conv_layer_size))
    batch_size = int(round(batch_size))
    dropout = dropout
    model = dc.models.GraphConvModel(
            len(tasks),
            batch_size=batch_size,
            dropout=dropout,
            mode='regression',
            graph_conv_layers=[graph_conv_layer_size]*graph_conv_depth,
            dense_layer_size=dense_layer_size,
            # configproto=config,
            **params
            )
    model.fit(train_gcn)

    # MAE
    valid_scores_mae = model.evaluate(val_gcn, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val_gcn, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val_gcn, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae

In [ ]:
# config = tf.compat.v1.ConfigProto()
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# Run the model 

neg_mae = []
mae = []
r2  = []
rmse = []
cov = matern32()
gp  = GaussianProcess(cov)
acq = Acquisition(mode='ExpectedImprovement')
tasks = ['pki']

gpgo = GPGO(gp, acq, hyper_model, params_dict)

# Adjust the number of iterations, e.g., max_iter=100 and init_eval=10.
max_iter = 10
init_evals = 1
gpgo.run(max_iter=max_iter, init_evals=init_evals)

In [ ]:
# hyperparameter tuning
"""
from deepchem.models import GCNModel

def hyper_model(dense_layer_size, graph_conv_layer_size, batch_size, dropout, nb_epoch, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    dense_layer_size = int(round(dense_layer_size))
    graph_conv_depth = 2 # two graph convolutional layers
    graph_conv_layer_size = int(round(graph_conv_layer_size))
    batch_size = int(round(batch_size))
    dropout = dropout
    model = GCNModel(
            n_tasks=1,
            graph_conv_layers=[graph_conv_layer_size]*graph_conv_depth,
            batch_size=batch_size,
            dropout=dropout,
            mode='regression',
            dense_layer_size=dense_layer_size,
            configproto=config,
            **params
            )
    model.fit(train)

    # MAE
    valid_scores_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae
"""